## pip install dfba  didn't work and we coudn't run the rest of the code

In [2]:
pip install dfba


  Using cached dfba-0.1.8.tar.gz (63 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'error'


  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [31 lines of output]
      C:\Users\harou\AppData\Local\Temp\pip-install-ww_umnw7\dfba_4e36544ea81c454d8526d5254f1d8fb9\versioneer.py:432: SyntaxWarning: invalid escape sequence '\s'
        ] = '''
      Traceback (most recent call last):
        File "c:\Users\harou\AppData\Local\Programs\Python\Python312\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 353, in <module>
          main()
        File "c:\Users\harou\AppData\Local\Programs\Python\Python312\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 335, in main
          json_out['return_val'] = hook(**hook_input['kwargs'])
                                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "c:\Users\harou\AppData\Local\Programs\Python\Python312\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 118, in 

In [3]:
from cobra.io import read_sbml_model

model = read_sbml_model('Files/iCGB21FR_updated.xml')

# Getting the exchange reaction names for glucose, ooxygen and lysine
for reaction in model.exchanges:
    if "_glc__D_e" in reaction.id or "_o2_e" in reaction.id or "_lys__L_e" in reaction.id:
        print(reaction.id)

EX_glc__D_e
EX_lys__L_e
EX_o2_e


In [4]:

from cobra.io import read_sbml_model
from dfba import DfbaModel, ExchangeFlux, KineticVariable
from dfba.plot.plotly import *

model = read_sbml_model('Files/iCGB21FR_updated.xml')
model.solver = "glpk"
dfba_model = DfbaModel(model)

biomass = KineticVariable("Biomass")
glucose = KineticVariable("Glucose")
oxygen = KineticVariable("Oxygen")
lysine = KineticVariable("Lysine")

dfba_model.add_kinetic_variables([biomass, glucose, oxygen, lysine])

mu = ExchangeFlux("Growth")
v_G = ExchangeFlux("EX_glc__D_e")
v_O = ExchangeFlux("EX_o2_e")
v_L = ExchangeFlux("EX_lys__L_e")

dfba_model.add_exchange_fluxes([mu, v_G, v_O, v_L])

dfba_model.add_rhs_expression("Biomass", mu * biomass)
dfba_model.add_rhs_expression("Glucose", v_G * 180.1559 * biomass / 1000.0)
dfba_model.add_rhs_expression("Oxygen", v_O * 16.0 * biomass / 1000.0)
dfba_model.add_rhs_expression("Lysine", v_L * 146.19 * biomass / 1000.0)

dfba_model.add_exchange_flux_lb("EX_glc__D_e", 10.5*(glucose/(0.0027+glucose)), glucose)
dfba_model.add_exchange_flux_lb("EX_o2_e", 15.0*(oxygen/(0.024+oxygen)), oxygen)
model.reactions.drain_biomass.lower_bound = 0.1   #[1/h]

dfba_model.add_initial_conditions(
    {
        "Biomass": 0.03, #(gDW/L)
        "Glucose": 15.5, #(g/L)
        "Oxygen": 1,     #(g/L)
        "Lysine" : 0     #(g/L)
    }
)
# simulating model across interval from 0.0h to 25.0h with outputs for plotting every 0.1h
concentrations, trajectories = dfba_model.simulate(0.0, 25.0, 0.01, ["Growth", "EX_glc__D_e", "EX_o2_e", "EX_lys__L_e"])

ModuleNotFoundError: No module named 'dfba'